## Simple agent the that randomly chooses allowable actions within a silhouette ('viable' actions)

Programming this agent as a baseline.
If we can bias the agent towards e.g. bigger blocks, that's a plus.

### input
Bitmap representation of each stimulus

### output
Output block placements as dataframe of same type as action dataframe used in other analyses (dfi) i.e.
targetName, blockNum, x,y,w,h

### stability
Blocks have to be placed on a 'floor', which includes two separated floors (to make a henge) 

In the experiment, unstable placements end the trial. We could:
a) allow the agent to make unstable placements, but end the trial when they do
b) not allow the agent to consider unstable placements
Here I go for option b, where possible actions don't include those that would fall

### action selection 
There are various ways to make a random agent:
a) enumerate all possible actions (all blocks in all locations), then uniformly select from these.
b) uniformly select a block, then uniformly select a location.
c) uniformly select a location, then uniformly select a block that fits there. 
d) uniformly select a block **and** uniformly select a location, reject if not possible.



In [1]:
from __future__ import division

import numpy as np
import os, sys
from PIL import Image
from os import listdir
from os.path import isfile, join

from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

from IPython.core.pylabtools import figsize, getfigs

import seaborn as sns

import random

from scipy.stats import norm
from IPython.display import clear_output

import numpy as np
import pandas as pd
import os
import json

import copy
import importlib

### Add Paths

## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'..','..')) ## use relative paths

## add helpers to python path
import sys
if os.path.join(proj_dir, 'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'stimuli'))

## custom helper modules
import separation_axis_theorem as sat
#import blockworld_helpers as utils
#import display_world as stability #may want to make a separate module for stability

def cls():
    os.system('cls' if os.name=='nt' else 'clear')


In [1]:
block_dims = [[1,2],
              [2,1],
              [2,2],
              [2,4],
              [4,2]] 




def simulate(stims,targets,target_maps,niter):
    
    columns = ['targetName','run','actionNum','discreteWorld','x','y','w','h']

    df = pd.DataFrame(columns=columns)
    
    for stim in stims:

        target_map = target_maps[stim]

        for run in range(0,10000):

            discrete_world = np.zeros([13,18])
            actionNum = 0
            exist_possible_actions = true

            while (exist_possible_actions & discrete_world!=target_map):

                floors = find_floors(discrete_world)

                x,y,w,h = select_action(floors)

                if (is_viable(discrete_world,x,y,w,h)):
                    discrete_world.place_block(x,y,w,h)

                    df.append(stim,run,actionNum,discrete_world,x,y,w,h)

                    actionNum += 1
    
    return df



def find_floors(current_structure):
    '''
    Finds the coordinates of empty squares directly above the current structure.
    There can be holes in the structure which allow block placements: include those.
    '''
    # dimensions to search in [5:13, 0:8]

    
        
        
        
return floors


def is_viable(current_structure,x,y,w,h):
    '''
    Check if action stays within silhouette and is stable?
    Check every square of block to see if it 
    a) coincides with silhouette 
    b) doesn't coincide with the current_construction (i.e. hit an overhang)
    '''


    return is_viable

IndentationError: expected an indented block (<ipython-input-1-9acec23bc623>, line 31)

In [162]:
#### Target maps: grab the bitmap representation of each stim

targets = ['hand_selected_004', 'hand_selected_005', 'hand_selected_006',
       'hand_selected_008', 'hand_selected_009', 'hand_selected_011',
       'hand_selected_012', 'hand_selected_016']

target_maps = {}

with open(os.path.abspath('../results/csv/targetMaps.txt')) as json_file:
    target_maps = json.load(json_file)
    

def check_overlap(x,y,w,h,world, mode='inside'):
    overlaps = False
    
    if mode == 'inside':
        overlaps = np.all(world[x:(x+w),y:(y+h)])
    elif mode == 'outside':
        overlaps = ~np.any(world[x:(x+w),y:(y+h)])
    else:
        return
    
    return overlaps

def check_stability(x,y,w,h,world):
    '''
    checks to see if block would be supported without falling using heuristics.
    Does not allow side-supported blocks, which are sometimes possible in the real experiments
    '''
    
    if y == 0: #if on the floor then will be stable
        return True
    else: #if greater than 1/2 of the block is supported then stable
        support = world[x:(x+w),y-1:y].astype(int)
        if np.sum(support) > w/2:
            return True
        # supports on both sides of long block
        elif (w == 4 & (sum(world[x:(x+2),y-1:y].astype(int)) > 1) & (sum(world[x+2:(x+w),y-1:y].astype(int)) > 1)):
            print('4:', world[x:(x+2),y-1:y].astype(int))
            return True
        else:
              return False

def find_positions(world, block, x_offset = 5):
    
    positions = []
    
    for i in range(world.shape[0]-block['width']):
            if (~np.any(world[i:i+block['width'],0])):
                positions.append({'x': i + x_offset,
                                  'y': 0})
                
    for j in range(1,world.shape[1]-block['height']+1):
        for i in range(world.shape[0]-block['width']):
            if ((~np.any(world[i:i+block['width'],j])) & np.any(world[i:i+block['width'],j-1])):
                positions.append({'x': i + x_offset,
                                  'y': j})
    return positions
                
    
            
block_dims = [{'width':1,
              'height':2},
              {'width':2,
               'height':1},
              {'width':2,
               'height':2},
              {'width':2,
               'height':4},
              {'width':4,
               'height':2}]

world_bounds = {'left': 5,
                'right': 13}

discrete_world = np.zeros([18,13]).astype(bool)
target_map = np.logical_not(np.array(target_maps['hand_selected_012']))

completed = False
tested_all_blocks = False



while (~completed & ~tested_all_blocks):
    
    placed = False
    
    random.shuffle(block_dims)
    
    b = 0
    while((b < len(block_dims)) & ~placed): #keep trying blocks until placed or none left
        
        #select next block from shuffled list
        block = block_dims[b]
        print(" "*0,'block:', block)
        
        # position-centric
        # enumerate all positions for that block
        positions = find_positions(discrete_world[5:13,0:8], block, x_offset=5)
        print(positions)

        random.shuffle(positions) # shuffle positions
        p = 0

        while(~placed & (p < len(positions))): #keep trying positions until placed or none left
            position = positions[p]
            print(" "*4,'position:', position)

            x_loc = position['x']
            y_loc = position['y']
            print(" "*4,'selected location x:', x_loc, 'y:', y_loc)

            # check if valid location
            # check if in silhouette
            within_silhouette = check_overlap(x_loc,y_loc,block['width'],block['height'], target_map, mode = 'inside')
            print(" "*4,'within silhouette:', within_silhouette)

            if within_silhouette:
                 # check if free in current world
                free_space = check_overlap(x_loc,y_loc,block['width'],block['height'], discrete_world, mode = 'outside')
                print(" "*5,'free space:', free_space)

                if free_space:

                    # check stability
                    stable = check_stability(x_loc, y_loc, block['width'], block['height'], discrete_world)
                    print(" "*4,'stable:', stable)

                    #if added:
                    if stable:
                        # add to world
                        discrete_world[x_loc:x_loc+block['width'],y_loc:y_loc+block['height']] = 1
                        # df.append(stim,run,actionNum,discrete_world,x,y,w,h)
                        print(np.rot90(discrete_world.astype(int)))
                        placed = True
                        completed = np.all(np.equal(discrete_world,target_map))
                        print('completed:',completed)
                    else:
                        p += 1 # check next position
                else:
                    p += 1 # check next position
            else:
                p += 1 # check next position
                
        if(p == len(positions)): # if no positions work
            b += 1 # check next block
            print('incrementing b to ',b)
            
    if b == len(block_dims):
        print('no viable blocks- giving up')
        tested_all_blocks = True

 block: {'width': 2, 'height': 4}
[{'y': 0, 'x': 5}, {'y': 0, 'x': 6}, {'y': 0, 'x': 7}, {'y': 0, 'x': 8}, {'y': 0, 'x': 9}, {'y': 0, 'x': 10}]
     position: {'y': 0, 'x': 5}
     selected location x: 5 y: 0
     within silhouette: False
     position: {'y': 0, 'x': 6}
     selected location x: 6 y: 0
     within silhouette: False
     position: {'y': 0, 'x': 8}
     selected location x: 8 y: 0
     within silhouette: True
      free space: True
     stable: True
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]]
completed: False
 block

In [ ]:
!git

In [124]:
np.all(np.equal(discrete_world,discrete_world))

True

In [260]:
print(np.rot90(target_map.astype(int)))

find_positions(target_map[5:13,0:8])


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0]]


[{'x': 6, 'y': 0}, {'x': 7, 'y': 0}, {'x': 12, 'y': 0}, {'x': 10, 'y': 2}]

[master 4d268e1] random agent
 1 file changed, 2148 insertions(+), 47 deletions(-)
Counting objects: 5, done.
Delta compression using up to 16 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 8.84 KiB | 2.21 MiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To github.com:cogtoolslab/block_construction.git
   5c20d0d..4d268e1  master -> master


In [4]:
#### Target maps: grab the bitmap representation of each stim

targets = ['hand_selected_004', 'hand_selected_005', 'hand_selected_006',
       'hand_selected_008', 'hand_selected_009', 'hand_selected_011',
       'hand_selected_012', 'hand_selected_016']

target_maps = {}

with open(os.path.abspath('../results/csv/targetMaps.txt')) as json_file:
    target_maps = json.load(json_file)

In [143]:
target_maps

{'hand_selected_004': [[True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [False,
   False,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [False,
   False,
   False,
   False,
   False,
   True,
   True,
   True,
   True,
   True,
   True,
   True,
   True],
  [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   True,
   True,
   True,
   True,
   True],
  [False,
   False,

# OLD
Everything below this was copied from stim generation for reference

### start with simple cases 

#### define inventory of block types, positioned at origin

In [ ]:
## sketch of blocklaying procedure:

## sample block size and place in leftmost, lowest position that you can and place in the array

## make sure that no blocks collide

## keep going until the summed area of all blocks equals or exceeds the total area of the arena

## fill these crevices

In [ ]:
importlib.reload(utils)
b1 = utils.Block(1,1)
b2 = utils.Block(2,2)
b3 = utils.Block(4,2)

In [ ]:
importlib.reload(utils)

def patch_for_block(b):
    return utils.get_patch(b.base_block.translate(b.base_block.verts,b.x,b.y),color=b.base_block.color)

def patches_for_floor(floor_blocks, xs):
    patches = []
    for (i, b) in enumerate(floor_blocks):
        patches.append(patch_for_block_here(b,xs[i],0))
    return patches

def drawFloor(floor_blocks, xs):
    utils.render_blockworld(patches_for_floor(floor_blocks, xs))
    
def patches_for_world(blocks):
    patches = []
    for (b) in blocks:
        patches.append(patch_for_block(b))
    return patches

def drawWorld(world):
    utils.render_blockworld(patches_for_world(world.blocks))

# World Class
class World:
    
    width = 8
    height = 8

    block_dims = [
        (1,1),
        (1,2),
        (2,1),
        (2,2),
        (2,4),
        (4,2),
        (4,4),
        (8,2)
        ]
    
    block_colors = [
        '#D33E43',
        '#29335C',
        '#EAEAEA',
        '#0F8B8D',
        '#2E3B44',
        '#E79598',
        '#8A8FA6',
        '#91CACB',
        '#B3B7BB',
        '#D33E43',
        '#EAEAEA'
    ]

    base_blocks = [utils.BlockType(w,h,color=c) for ((w,h),c) in list(zip(sorted(block_dims),block_colors[0:len(block_dims)]))] # Block types should be in order from left to right, thinest to thickest, shortest to tallest
    block_widths = list(map(lambda b: b.width, base_blocks))
    
    def __init__(self):
        
        # Actual blocks present in world
        
        self.block_map = np.zeros((World.width, World.height), dtype=int) #bitmap for placement of blocks
        
        self.blocks = []
        
        self.full = False
        
    def check_full(self):
        '''
        Checks to see whether the World contains any empty space by summing block_map
        '''
        if not self.full:
            isFull = (sum(sum(self.block_map)) == width*height)
            self.full = isFull
            return isFull
        else:
            return True
      
    def fill_floor(self, floor_space):
        '''
        Fills a 'floor', a level horizontal surface, with blocks.
        Input: Lexicon of blocks- np arrays with 5 coordinates; length of available floor space
        Output: List of blocks that can be used to fill the floor space with no gaps
        '''
        
        floor_blocks = []
        floor_block_widths = []
        viable_block_widths = copy.deepcopy(self.block_widths)
        remaining_space = floor_space
        while remaining_space > 0:
            i = np.random.randint(0,len(viable_block_widths))
            if self.block_widths[i] <= remaining_space:
                floor_blocks.append([self.base_blocks[i],floor_space-remaining_space])
                floor_block_widths.append(self.block_widths[i])
                remaining_space -= self.block_widths[i]
            else:
                viable_block_widths.pop()
        return(floor_blocks)

    def fill_floor_here(self, lvl, left, right):
        '''
        Fills a 'floor', a level horizontal surface, with blocks.
        Input: Lexicon of blocks- np arrays with 5 coordinates; length of available floor space
        Output: List of blocks that can be used to fill the floor space with no gaps
        '''
        
        floor_space = right-left
        floor_blocks = []
        floor_block_x_location = left
        
        viable_block_widths = copy.deepcopy(self.block_widths)
        viable_blocks = copy.deepcopy(self.base_blocks)
        
        remaining_height = World.height - lvl
        remaining_space = floor_space
        
        while remaining_space > 0:
            i = np.random.randint(0,len(viable_blocks))
            base_block = self.base_blocks[i]
            if base_block.width <= remaining_space and base_block.height <= remaining_height: 
                b = utils.Block(self.base_blocks[i],floor_block_x_location,lvl)
                floor_block_x_location += b.width
                floor_blocks.append(b)
                self.blocks.append(b)
                remaining_space -= b.width
            else:
                viable_blocks.pop()
        self._update_map_with_floor_blocks(floor_blocks)

    def _update_map_with_floor_blocks(self, floor_blocks):
        for (i, b) in enumerate(floor_blocks):
            self.block_map[self.height-(b.y+b.height): self.height-b.y, b.x:(b.x+b.width)] = 1  
            #self.block_map[self.height-(lvl+b.height): self.height-lvl, floor_block_xs[i]:floor_block_xs[i+1]] = 1  
        #print(self.block_map)
    
    def fill_world(self):
        '''
        Semi-randomly fills world with blocks, adding a 'floor' of blocks to the next available flat surface 
        '''
        lvl = 0 #Start at bottom and work up
        while lvl <= World.height - 1: # Until at top
            #find floor
            while self.block_map[World.height - lvl - 1].all() and lvl < World.height: # check if level is full or reached top
                lvl += 1
            if lvl == World.height:
                    break
            left = 0
            while self.block_map[World.height - lvl - 1][left] == 1:
                left += 1
            right = left
            while right < World.width and self.block_map[World.height - lvl - 1][right] == 0:
                right += 1
            #print('fill_world_here: ' + str((lvl, left, right)))
            self.fill_floor_here(lvl, left, right)
        
        drawWorld(self)
    
    

In [ ]:
importlib.reload(utils)
w = utils.World()
w.fill_world(render=True)

In [ ]:
importlib.reload(utils)
w = utils.World()
w.add_block(4,2,1,4)
w.add_block(2,4,2,0)
utils.draw_world(w)

In [ ]:
# successively 'Jenga' blocks
importlib.reload(utils)
w = utils.World()
w.fill_world()
world_stages = [w]
# loop

# try removing blocks until one removed or no blocks left

utils.draw_world(w)
for j in range(0,5):
    i = 0;
    block_removed = False
    while not block_removed:
        #block_number = random_block_order.pop
        (block_removed, w2) = w.jenga_block(i)
        if block_removed:
            world_stages.append(w2)
            w = w2
            utils.draw_world(w)
        else:
            i += 1;


In [ ]:
importlib.reload(utils)
w = utils.World()
w.fill_world()
json_obj = w.save_to_json()
json_obj

In [ ]:
w2 = utils.World()
w2.populate_from_json(json_obj)
utils.draw_world(w2)
utils.draw_world(w)

In [ ]:
importlib.reload(utils)
w = utils.World()
a = w.add_block(4,2,4,4)
b = w.add_block(2,4,4,0)
utils.draw_world(w)
print('\n',
    'a above b: ', a.above(b), '\n', 
    'a below b: ', a.below(b), '\n', 
    'a left b: ', a.leftof(b), '\n', 
    'a right b: ', a.rightof(b)
)

print('\n',
    'b above a: ', b.above(a), '\n', 
    'b below a: ', b.below(a), '\n', 
    'b left a: ', b.leftof(a), '\n', 
    'b right a: ', b.rightof(a)
)

print('\n',
    'a touching b: ', a.touching(b), '\n',
    'b touching a: ', b.touching(a))

print('\n',
    'a abs_overlap b (horizontal): ', a.abs_overlap(b), '\n',
    'b abs_overlap a (horizontal): ', b.abs_overlap(a), '\n',
    'a abs_overlap b (vertical): ', a.abs_overlap(b, horizontal_overlap=False), '\n',
    'b abs_overlap a (vertical): ', b.abs_overlap(a, horizontal_overlap=False))


print('\n',
    'a partially supported by b: ', a.partially_supported_by(b), '\n',
    'b partially supported by a: ', b.partially_supported_by(a))

print('\n',
    'a completely supported by b: ', a.completely_supported_by(b), '\n',
    'b completely supported by a: ', b.completely_supported_by(a))

In [ ]:
importlib.reload(utils)
w2 = utils.World()
w2.add_block(1,2,2,0)
w2.add_block(2,2,2,2)
w2.add_block(2,2,4,0)
w2.add_block(2,2,4,2)
w2.add_block(1,2,7,0)
w2.add_block(1,2,7,2)
w2.add_block(4,2,4,4)
w2.add_block(4,2,4,6)
w2.add_block(2,1,2,4)
w2.add_block(2,1,2,5)
utils.draw_world(w2)



block_array_x = np.array(w2.blocks)[None,:]
block_array_y = np.array(w2.blocks)[:,None]

#block_array_x = np.expand_dims(w2.blocks, axis=0)
#block_array_y = np.expand_dims(w2.blocks, axis=1)

above_function = np.vectorize(utils.Block.above)
aboves = above_function(block_array_x, block_array_y)
print("above \n",aboves)

below_function = np.vectorize(utils.Block.below)
belows = below_function(block_array_x, block_array_y)
print("below \n", belows)

partial_support_function = np.vectorize(utils.Block.partially_supported_by)
partial_support = partial_support_function(block_array_x, block_array_y)
print("partial support \n", partial_support)

In [ ]:
def jenga_blocks(w,n):

    for j in range(0,n):
        i = 0;
        block_removed = False
        while not block_removed:
            #block_number = random_block_order.pop
            (block_removed, w2) = w.jenga_block(i)
            if block_removed:
                w = w2
            else:
                i += 1;
    return w

In [ ]:
importlib.reload(utils)
w = utils.World()
w.fill_world()
w = jenga_blocks(w,5)
utils.draw_world(w)

block_array_x = np.array(w.blocks)[None,:]
block_array_y = np.array(w.blocks)[:,None]

partial_support_function = np.vectorize(utils.Block.partially_supported_by)
partial_support = partial_support_function(block_array_x, block_array_y)
print("partial support \n")

im = Image.fromarray(partial_support)

im.resize((100,100))

In [ ]:
importlib.reload(utils)
w = utils.World(
    block_dims = [  (1,2),
                    (2,1),
                    (2,2),
                    (2,4),
                    (4,2)
                    ])
w.fill_world()
w = jenga_blocks(w,5)
utils.draw_world(w)

In [ ]:
world_json = w.save_to_json()

f = open("../interesting_structures.json", "a")
f.write("\n")
f.write(world_json)
f.close()

In [ ]:
importlib.reload(stability)
filepath = "../interesting_structures.json"
with open(filepath) as fp:
    for i,blocks in enumerate(fp):
        w = utils.World()
        #w = utils.World(block_colors = ['#000000','#000000','#000000','#000000','#000000','#000000','#000000','#000000'])
        w.populate_from_json(blocks)
        utils.draw_world(w)
        print(stability.test_world_stability(w))

In [ ]:
importlib.reload(utils)
# generate world
w, block_dict = utils.generate_random_world()
# save world
utils.save_world(block_dict)

In [ ]:
importlib.reload(utils)
from IPython.display import clear_output
num_worlds = 100
for i in np.arange(num_worlds):
    
    # generate world
    w, block_dict = utils.generate_random_world(remove_num_blocks=5, 
                                                block_dims = [
                                                            (1,2),
                                                            (2,1),
                                                            (2,2),
                                                            (2,4),
                                                            (4,2)
                                                            ])   
    # save world
    utils.save_world(block_dict,path_to_dump = './test')
    clear_output(wait=True)


In [ ]:
image_paths = [f for f in listdir('./test_render') if isfile(join('./test_render/', f))]    
fig=plt.figure(figsize=(20, 80))
columns = 5
rows = 20
for i in range(1, columns*rows +1):
    img = mpimg.imread(join('./test_render',image_paths[i-1+99]))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
image_paths = [f for f in listdir('./sampled_worlds_render') if isfile(join('./sampled_worlds_render/', f))]    
fig=plt.figure(figsize=(20, 80))
columns = 5
rows = 20
for i in range(1, columns*rows +1):
    img = mpimg.imread(join('./sampled_worlds_render',image_paths[i-1+200]))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
w, block_dict = utils.generate_random_world()
w_id = utils.generate_worldId()
print(w_id)
utils.save_world_json(block_dict,worldId = w_id)
utils.save_world_render(block_dict,worldId = w_id)
importlib.reload(stability)
print(stability.test_world_stability(w))

In [ ]:
importlib.reload(utils)
from IPython.display import clear_output
num_worlds = 100
stable_list = []
for i in np.arange(num_worlds):
    # generate world
    w, block_dict = utils.generate_random_world(remove_num_blocks=5, 
                                                block_dims = [
                                                            (1,2),
                                                            (2,1),
                                                            (2,2),
                                                            (2,4),
                                                            (4,2)
                                                            ])   

    
    # save world
    utils.save_world(block_dict,path_to_dump = './test')
    clear_output(wait=True)

        
        


In [ ]:
import json
importlib.reload(stability)
image_paths = np.array([f for f in listdir('./test_render') if isfile(join('./test_render/', f))])
json_paths = np.array([f for f in listdir('./test_json') if isfile(join('./test_json/', f))])
image_paths.sort()
json_paths.sort()

fig=plt.figure(figsize=(20, 80))
columns = 5
rows = 20
for i in range(1, columns*rows +1):
    img = mpimg.imread(join('./test_render', image_paths[i-1]))
    w = utils.World()
    J = open(join('./test_json', json_paths[i-1])).read()
    #J = json.loads(open(os.path.join(path_to_dump,'blockworld_area{}_num{}.js'.format(area_filled,num_blocks)),mode='r').read())
    w.populate_from_json(J)
    fig.add_subplot(rows, columns, i, title=stability.test_world_stability(w, Y_SHIFT=1.1))
    plt.imshow(img)
plt.show()

In [ ]:
utils.draw_world({"blocks": [{"x": 0, "y": 0, "height": 2, "width": 2}, {"x": 4, "y": 0, "height": 2, "width": 4}, {"x": 1, "y": 2, "height": 2, "width": 4}, {"x": 0, "y": 4, "height": 2, "width": 2}, {"x": 2, "y": 4, "height": 2, "width": 4}, {"x": 0, "y": 6, "height": 1, "width": 2}, {"x": 2, "y": 6, "height": 2, "width": 4}, {"x": 0, "y": 7, "height": 1, "width": 2}]})